# Qwen3 Embedding on AWS Trainium with Optimum Neuron

This guide explains how to convert, load, and use Qwen Embedding models on AWS Trainium and Inferentia2 using Optimum Neuron. The Qwen3 Embedding model series is the latest proprietary model of the Qwen family, specifically designed for text embedding and ranking tasks. Building upon the dense foundational models of the Qwen3 series, it provides a comprehensive range of text embeddings and reranking models in various sizes (0.6B, 4B, and 8B). The Qwen3 Embedding series offer support for over 100 languages, thanks to the multilingual capabilites of Qwen3 models. 

## Prerequisite: Setup Environment 

You can run this notebook on a AWS EC2 instance with the HF DLAMI. To create an instance with the DLAMI, you can follow the [EC2 Setup guide](https://huggingface.co/docs/optimum-neuron/en/ec2-setup). Alternatively if you are on a AWS Trainium and Inferentia instance, you can manually install `optimum-neuron` using the steps in the [manual installation guide](https://huggingface.co/docs/optimum-neuron/en/ec2-setup#alternative-manual-installation). 

This guide is written using a `trn2.3xlarge` AWS Trainium2 instance. But you can use the same code to run the model using a AWS Inferentia2 instance like `inf2.48xlarge`.

## Compile Qwen Embedding Models for AWS Trainium
First, you need to convert the model to a format compatible with AWS Trainium and Inferentia2. You can compile Qwen3 Embedding models with Optimum Neuron using the `optimum-cli` or `NeuronModelForEmbedding` class. Below you will find an example for both approaches. 

In the below example, we illustrate this using [Qwen3 Embedding 8B](https://huggingface.co/Qwen/Qwen3-Embedding-8B) but you can follow the same steps for the 0.8B and the 4B versions of the embedding models.

### Option A: Compile using the `NeuronModelForEmbedding` class
Here we will use the `NeuronModelForEmbedding` class, which can convert Qwen3 Embedding models to a format compatible with AWS Trainium and Inferentia2 or load already converted models. When exporting models with `NeuronModelForEmbedding` you need to define the the `sequence_length` and `batch size` in the neuron config.

In [ ]:
from optimum.neuron import NeuronModelForEmbedding

model_id = "Qwen/Qwen3-Embedding-8B"
neuron_model_dir = "qwen_embedding_8B_tp4"

# If you are using a AWS Inferentia2 instance and use 'tensor_parallel_size=4', you should set the following environment variable as well.
#import os
#os.environ["LOCAL_WORLD_SIZE"] = '4'

neuron_config = NeuronModelForEmbedding.get_neuron_config(
        model_id, batch_size=2, sequence_length=1024, tensor_parallel_size=4
    )

neuron_model = NeuronModelForEmbedding.export(
        model_id=model_id, neuron_config=neuron_config, load_weights=False
    )

# Save model to disk
neuron_model.save_pretrained(neuron_model_dir)

### Option B: Compile using the `optimum-cli` tool
Here we will use the `optimum-cli` tool to convert the model. Similar to the `NeuronModelForEmbedding` we need to define our sequence length and batch size. The `optimum-cli` will automatically convert the model to a format compatible with AWS Trainium and Inferentia2 and save it to the specified output directory.

In [ ]:
! optimum-cli export neuron --model Qwen/Qwen3-Embedding-8B --batch_size 2 --sequence_length 1024 --auto_cast matmul --instance_type trn2 --tensor_parallel_size 4 qwen_embedding_8B_tp4/

## Load compiled Qwen3 Embedding model and run inference
Once we have a compiled the model, for loading the model we can use the `NeuronModelForEmbedding` class to load the model and run inference.
In the below example, we first compute embeddings for two queries and documents; and then compute the similarity score for the queries and documents.

In [ ]:
import torch
from torch import Tensor
import torch.nn.functional as F

from transformers import AutoTokenizer
from optimum.neuron import NeuronModelForEmbedding

#set this to the path that has the compiled model from above
model_id_or_path = neuron_model_dir


def last_token_pool(last_hidden_states: Tensor, attention_mask: Tensor) -> Tensor:
    left_padding = attention_mask[:, -1].sum() == attention_mask.shape[0]
    if left_padding:
        return last_hidden_states[:, -1]
    else:
        sequence_lengths = attention_mask.sum(dim=1) - 1
        batch_size = last_hidden_states.shape[0]
        return last_hidden_states[torch.arange(batch_size, device=last_hidden_states.device), sequence_lengths]


# Load model and tokenizer
model = NeuronModelForEmbedding.from_pretrained(model_id_or_path)
tokenizer = AutoTokenizer.from_pretrained(model_id, padding_side="left")

# Input text to embed
queries = [
    "What is the capital of China?",
    "Explain gravity"
]
# No need to add instruction for retrieval documents
documents = [
    "The capital of China is Beijing.",
    "Gravity is a force that attracts two bodies towards each other. It gives weight to physical objects and is responsible for the movement of planets around the sun."
]


# Tokenize the input texts
queries_tokens = tokenizer(
    queries,
    padding=True,
    truncation=True,
    max_length=8192,
    return_tensors="pt",
)
documents_tokens = tokenizer(
    documents,
    padding=True,
    truncation=True,
    max_length=8192,
    return_tensors="pt",
)

outputs = model(**queries_tokens)
queries_embeddings = last_token_pool(outputs, queries_tokens['attention_mask'])

outputs = model(**documents_tokens)
documents_embeddings = last_token_pool(outputs, documents_tokens['attention_mask'])

# normalize embeddings and compute similarity scores
queries_embeddings = F.normalize(queries_embeddings, p=2, dim=1)
documents_embeddings = F.normalize(documents_embeddings, p=2, dim=1)

scores = (queries_embeddings @ documents_embeddings.T)
print("Similarity Scores -->", scores.tolist())